In [1]:
############################## package install #################################
if (!require(ade4)) { install.packages("ade4") ; library(ade4) } 
library(BBmisc)
library(reshape2)
library(dplyr)
library(xgboost)
library(caret)
library(mlr)
library(DMwR)

Loading required package: ade4
Warning message:
"package 'ade4' was built under R version 3.4.1"Warning message:
"package 'BBmisc' was built under R version 3.4.1"Warning message:
"package 'reshape2' was built under R version 3.4.1"Warning message:
"package 'dplyr' was built under R version 3.4.1"
Attaching package: 'dplyr'

The following objects are masked from 'package:BBmisc':

    coalesce, collapse

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'xgboost' was built under R version 3.4.1"
Attaching package: 'xgboost'

The following object is masked from 'package:dplyr':

    slice

Warning message:
"package 'caret' was built under R version 3.4.1"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.4.1"Warning message:
"package 'mlr' was built under R version

In [2]:

################################### read.data ###################################
data_Train_All = read.table('base_MICE_Tlink_regular_scale_AllTrain.csv', header = T, sep = ',', stringsAsFactors= T)
# data_Answer <- read.table('num1_random_data_smotetomek_Answer.csv', header = T, sep = ',', stringsAsFactors= T)
str(data_Train_All)
# index = c('TARGET', 'CPT_LNIF_CNT','SPART_LNIF_CNT','TOT_LNIF_AMT','SPTCT_OCCR_MDIF','CRDT_CARD_CNT','CTCD_OCCR_MDIF','LT1Y_PEOD_RATE','FMLY_TOT_PREM','LT1Y_CTLT_CNT','FMLY_PLPY_CNT','MOBL_FATY_PRC','NUM_DAY_SUSP', 'TLFE_UNPD_CNT','LT1Y_MXOD_AMT','vec.prop','AGE_9.0','PAYM_METD_0.0')
str(data_Train_Val)
data_Train_All = data_Train_All[,!(colnames(data_Train_All)%in%c('X'))]
str(data_Train_All)


'data.frame':	99032 obs. of  147 variables:
 $ X                     : int  0 1 2 3 4 5 6 7 8 9 ...
 $ TARGET                : num  0 0 0 1 0 0 0 0 0 0 ...
 $ BNK_LNIF_CNT          : num  1.19 1.19 0 0 4.76 ...
 $ CPT_LNIF_CNT          : num  0 0 0.333 0.998 0 ...
 $ SPART_LNIF_CNT        : num  0 0 1.57 2.1 0 ...
 $ ECT_LNIF_CNT          : num  0 0 1.39 1.05 0 ...
 $ TOT_LNIF_AMT          : num  16 17 16.5 15.6 16.9 ...
 $ TOT_CLIF_AMT          : num  16 0 16 14.9 16.5 ...
 $ BNK_LNIF_AMT          : num  16 17 0 0 16.9 ...
 $ CPT_LNIF_AMT          : num  0 0 14.9 14.9 0 ...
 $ CRDT_OCCR_MDIF        : num  1 0 1 1 1 1 11 1 4 1 ...
 $ SPTCT_OCCR_MDIF       : num  0 0 3 3 0 1 11 0 0 1 ...
 $ CRDT_CARD_CNT         : num  2 2 4 4 1 4 2 2 5 3 ...
 $ CTCD_OCCR_MDIF        : num  2 11 11 6 9 11 11 11 11 3 ...
 $ CB_GUIF_CNT           : num  3 0 0 0 0 0 1 0 0 0 ...
 $ CB_GUIF_AMT           : num  19.9 0 0 0 0 ...
 $ CUST_JOB_INCM         : num  17.8 17.8 0 0 17.7 ...
 $ HSHD_INFR_INCM        :

ERROR: Error in str(data_Train_Val): 객체 'data_Train_Val'를 찾을 수 없습니다


In [4]:
##########################################################################################
### F measure Function

F_measure = function(y, pred_y){
  result <- table(y, pred_y)
  Precision <- result[4] / (result[3] + result[4])
  Recall <- result[4] / (result[2] + result[4])
  f.value <- 2 * (Precision * Recall) / (Precision + Recall)
  return(f.value)
  print(f.value)
}

In [11]:
###########################################################################################################
###########################################################################################################

All_proc = function(data_Train_Val,pred_y_test, m){
  
  for(i in 1:m){#부스트렙횟수
    ###########################
    data_Train_All$TARGET = as.factor(data_Train_All$TARGET)
      
    index_y_1 = (1:nrow(data_Train_All))[data_Train_All['TARGET'] == 1]
    index_y_1 = sample(index_y_1)
    index_y_0 = (1:nrow(data_Train_All))[data_Train_All['TARGET'] == 0]
    index_y_0 = sample(index_y_0)

    vali_size_1  = round(2019*0.042)
    vali_size_0 = 2019 - round(2019*0.042)

    train_size_1 = length(index_y_1) - vali_size_1
    train_size_0 = length(index_y_1) - vali_size_0

    index_vali = c(index_y_1[1:vali_size_1], index_y_0[1:vali_size_0])
    index_train = c(index_y_1[(vali_size_1+1):(vali_size_1+train_size_1)], index_y_0[(vali_size_0+1):(vali_size_0+train_size_0)])

    data_Train = data_Train_All[index_train,]
    data_vali1 = data_Train_All[index_vali,]
    
    ############################### XGBOOST Tuning #################################
    
    # Step1: task and learner
    
    train.task <- makeClassifTask(data = data_Train, target = 'TARGET')
    
    lrn.main <- makeLearner('classif.xgboost', predict.type = "response")
    
    lrn.main$par.vals <- list( objective="binary:logistic", eval_metric="error", booster="gbtree", nrounds=5)
    
    ps.set <- makeParamSet(
      makeIntegerParam("max_depth",lower = 8, upper = 20), 
      makeIntegerParam("min_child_weight",lower = 1, upper = 10)
    )
    
    
    # Step2: search and validation
    ctrl <- makeTuneControlRandom(maxit = 20)
    rdesc <- makeResampleDesc('CV', iters = 5L, stratify = T)
    
    
    # Step3: tune
    tune.lrn.main <- tuneParams(lrn.main, task = train.task, resampling = rdesc,
                                par.set = ps.set, control = ctrl,
                                measures = acc)
    
    
    lrn.main <- setHyperPars(lrn.main, par.vals = tune.lrn.main$x)
    
    
    #################################################################################
    lrn.main$par.vals <- list(objective="binary:logistic", eval_metric="error", booster="gbtree", nrounds=5,
                              max_depth = lrn.main$par.vals$max_depth, min_child_weight = lrn.main$par.vals$min_child_weight)
    
    
    ps.set <- makeParamSet(
      makeNumericParam("gamma", lower=0.001, upper=0.1)
    )
    
    
    # Step2: search and validation
    ctrl <- makeTuneControlRandom(maxit = 20)
    rdesc <- makeResampleDesc('CV', iters = 5L, stratify = T)
    
    
    # Step3: tune
    tune.lrn.main <- tuneParams(lrn.main, task = train.task, resampling = rdesc,
                                par.set = ps.set, control = ctrl,
                                measures = acc)
    
    
    lrn.main <- setHyperPars(lrn.main, par.vals = tune.lrn.main$x)
    
    
    
    #################################################################################
    lrn.main$par.vals <- list(objective="binary:logistic", eval_metric="error", booster="gbtree", nrounds=5,
                              max_depth = lrn.main$par.vals$max_depth, min_child_weight = lrn.main$par.vals$min_child_weight,
                              gamma = lrn.main$par.vals$gamma
    )
    
    
    ps.set <- makeParamSet(
      makeNumericParam("subsample",lower = 0.6, upper = 0.95),
      makeNumericParam("colsample_bytree",lower = 0.5, upper = 0.75)
    )
    
    
    # Step2: search and validation
    ctrl <- makeTuneControlRandom(maxit = 20)
    rdesc <- makeResampleDesc('CV', iters = 5L, stratify = T)
    
    
    # Step3: tune
    tune.lrn.main <- tuneParams(lrn.main, task = train.task, resampling = rdesc,
                                par.set = ps.set, control = ctrl,
                                measures = acc)
    
    
    lrn.main <- setHyperPars(lrn.main, par.vals = tune.lrn.main$x)
    
    
    
    #################################################################################
    lrn.main$par.vals <- list(objective="binary:logistic", eval_metric="error", booster="gbtree", nrounds=5,
                              max_depth = lrn.main$par.vals$max_depth, min_child_weight = lrn.main$par.vals$min_child_weight,
                              gamma = lrn.main$par.vals$gamma, 
                              subsample = lrn.main$par.vals$subsample, colsample_bytree = lrn.main$par.vals$colsample_bytree
    )
    
    
    ps.set <- makeParamSet(
      makeNumericParam("eta",lower = 0.01, upper = 0.3)
    )
    
    # Step2: search and validation
    ctrl <- makeTuneControlRandom(maxit = 20)
    rdesc <- makeResampleDesc('CV', iters = 5L, stratify = T)
    
    
    # Step3: tune
    tune.lrn.main <- tuneParams(lrn.main, task = train.task, resampling = rdesc,
                                par.set = ps.set, control = ctrl,
                                measures = acc)
    
    
    lrn.main <- setHyperPars(lrn.main, par.vals = tune.lrn.main$x)
    
    
    
    ############################## Train and predict ################################
    
    # Step1: predictions on main
    
    lrn.main$par.vals <- list(objective="binary:logistic", eval_metric="error", booster="gbtree", nrounds = 100, nthread = 4,
                              max_depth = lrn.main$par.vals$max_depth, min_child_weight = lrn.main$par.vals$min_child_weight,
                              gamma = lrn.main$par.vals$gamma, 
                              subsample = lrn.main$par.vals$subsample, colsample_bytree = lrn.main$par.vals$colsample_bytree,
                              eta = lrn.main$par.vals$eta
    )
    
    global.main.train <- train(lrn.main, train.task)
    
    tt_test1 <- predict(global.main.train, newdata =  data_vali1[, -1])
    
    count=1
    for(j in 1:nrow(data_Train_All)){
      if(j%in%index_vali)
      {
        pred_y_test1[j,i] <- tt_test1$data$response[count]
        count= count+1
      }
      
    }
    
    
    print(i)
    print(table(pred_y_test1[,i]))
    print(table(data_Train_All[index_vali,]$TARGET, pred_y_test1[!is.na(pred_y_test1[,i]),i]))
    print(F_measure(data_Train_All[index_vali,]$TARGET, pred_y_test1[!is.na(pred_y_test1[,i]),i]))
    
  }  
  
  assign('pred_y_test1', pred_y_test1, envir = .GlobalEnv)
  #assign('pred_y_test2', pred_y_test2, envir = .GlobalEnv)
  
}

In [ ]:
###########################################################################################################
###########################################################################################################
m = 30 # ?삁痢≫슏?닔
pred_y_test1 = matrix(rep(NA, m * nrow(data_Train_All)), ncol = m)
#pred_y_test2 = matrix(rep(0, m * nrow(data_Test2)), ncol = m)

All_proc(data_Train_All,pred_y_test, m)

[1] 85
[1] 1934
[1] 147
[1] 147


[Tune] Started tuning learner classif.xgboost for parameter set:
                    Type len Def  Constr Req Tunable Trafo
max_depth        integer   -   - 8 to 20   -    TRUE     -
min_child_weight integer   -   - 1 to 10   -    TRUE     -
With control class: TuneControlRandom
Imputation value: -0
[Tune-x] 1: max_depth=19; min_child_weight=9


[1]	train-error:0.150105 
[2]	train-error:0.137898 
[3]	train-error:0.130841 
[4]	train-error:0.127980 
[5]	train-error:0.121877 
[1]	train-error:0.148169 
[2]	train-error:0.132723 
[3]	train-error:0.123570 
[4]	train-error:0.117468 
[5]	train-error:0.112128 
[1]	train-error:0.148198 
[2]	train-error:0.137135 
[3]	train-error:0.128552 
[4]	train-error:0.118062 
[5]	train-error:0.111959 
[1]	train-error:0.145091 
[2]	train-error:0.132698 
[3]	train-error:0.128694 
[4]	train-error:0.122402 
[5]	train-error:0.116301 
[1]	train-error:0.145663 
[2]	train-error:0.134223 
[3]	train-error:0.121449 
[4]	train-error:0.118398 
[5]	train-error:0.110772 


[Tune-y] 1: acc.test.mean=0.847; time: 0.0 min
[Tune-x] 2: max_depth=19; min_child_weight=8


[1]	train-error:0.148388 
[2]	train-error:0.135228 
[3]	train-error:0.130460 
[4]	train-error:0.123975 
[5]	train-error:0.120160 
[1]	train-error:0.146072 
[2]	train-error:0.127765 
[3]	train-error:0.123188 
[4]	train-error:0.116133 
[5]	train-error:0.112700 
[1]	train-error:0.147053 
[2]	train-error:0.132176 
[3]	train-error:0.121877 
[4]	train-error:0.118444 
[5]	train-error:0.109479 
[1]	train-error:0.143947 
[2]	train-error:0.129838 
[3]	train-error:0.123928 
[4]	train-error:0.116683 
[5]	train-error:0.110010 
[1]	train-error:0.142421 
[2]	train-error:0.128694 
[3]	train-error:0.121449 
[4]	train-error:0.111535 
[5]	train-error:0.107150 


...

[Tune-y] 17: acc.test.mean=0.848; time: 0.0 min
[Tune-x] 18: subsample=0.742; colsample_bytree=0.506


[1]	train-error:0.158085 
[2]	train-error:0.142258 
[3]	train-error:0.133677 
[4]	train-error:0.126240 
[5]	train-error:0.120900 
[1]	train-error:0.157543 
[2]	train-error:0.135419 
[3]	train-error:0.124928 
[4]	train-error:0.125310 
[5]	train-error:0.115010 
[1]	train-error:0.170862 
[2]	train-error:0.147788 
[3]	train-error:0.136918 
[4]	train-error:0.129291 
[5]	train-error:0.122616 
[1]	train-error:0.166635 
[2]	train-error:0.143947 
[3]	train-error:0.130601 
[4]	train-error:0.124500 
[5]	train-error:0.120305 
[1]	train-error:0.192220 
[2]	train-error:0.147597 
[3]	train-error:0.137300 
[4]	train-error:0.133105 
[5]	train-error:0.123951 


[Tune-y] 18: acc.test.mean=0.843; time: 0.0 min
[Tune-x] 19: subsample=0.718; colsample_bytree=0.727


[1]	train-error:0.159992 
[2]	train-error:0.140732 
[3]	train-error:0.131960 
[4]	train-error:0.128528 
[5]	train-error:0.125286 
[1]	train-error:0.178142 
[2]	train-error:0.145718 
[3]	train-error:0.128171 
[4]	train-error:0.121877 
[5]	train-error:0.116918 
[1]	train-error:0.165904 
[2]	train-error:0.140923 
[3]	train-error:0.129672 
[4]	train-error:0.121281 
[5]	train-error:0.116133 
[1]	train-error:0.156721 
[2]	train-error:0.142231 
[3]	train-error:0.129075 
[4]	train-error:0.125643 
[5]	train-error:0.117826 
[1]	train-error:0.162471 
[2]	train-error:0.139588 
[3]	train-error:0.132723 
[4]	train-error:0.130053 
[5]	train-error:0.125667 


[Tune-y] 19: acc.test.mean=0.845; time: 0.0 min
[Tune-x] 20: subsample=0.741; colsample_bytree=0.575


In [56]:
str(data_Train_Val)
str(data_Test1)
str(data_Test2)

ERROR: Error in str(data_Train_Val): 객체 'data_Train_Val'를 찾을 수 없습니다


In [57]:
# test1
#각각 F값구하기 and f score 평균
MEAN = c()
for(i in 1:ncol(pred_y_test1))
{
  fs = F_measure(data_Train_All[!is.na(pred_y_test1[,i]),]$TARGET, pred_y_test1[!is.na(pred_y_test1[,i]),i]) %>%print
  MEAN = rbind(MEAN,fs)
}  

[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA
[1] NA


In [ ]:
#each bootstrap f score 평균
sum(MEAN)/ncol(pred_y_test1)
# write.csv(pred_y_test1,'smote_default_xgb.csv')

In [ ]:
max_p1 = 0;max_F1 = 0
fscore = c()
for(i in seq(0.01,0.99,0.01)){
  
  selected_count = apply(pred_y_test1,1,function(c)sum(!is.na(c)))
  prediction_test1 = rowSums(pred_y_test1-1,na.rm=TRUE)
  prediction_test1 = prediction_test1/selected_count
  prediction_test1 = 1 * (prediction_test1 >= i)
  prediction_test1
  
  print(i)
  print(table(data_Train_All$TARGET, prediction_test1))
  print(F_measure(data_Train_All$TARGET, prediction_test1))
  fscore = rbind(fscore,F_measure(data_Train_All$TARGET, prediction_test1))
  
  if(max_F1 < F_measure(data_Train_All$TARGET, prediction_test1)){
    max_F1 = F_measure(data_Train_All$TARGET, prediction_test1)
    max_p1 = i}
}

In [ ]:
x = seq(0.01,0.99,0.01)
y = fscore
plot(x,y) #최적의 p값

In [ ]:
max(y) #최고의 fscore

In [ ]:
###########################부스트렙 테이블 저장하기 ######################
Bootstraped = cbind(data_Train_All$TARGET,pred_y_test1)
# write.csv(Bootstraped,'1.csv')

selected_count = apply(pred_y_test1,1,function(c)sum(!is.na(c)))
prediction_test1 = apply(pred_y_test1-1,1,sum(is.rm=T))
prediction_test1 = prediction_test1/selected_count
prediction_test1 = 1 * (prediction_test1 >= max_p1)
prediction_test1

table(data_Test1$TARGET, prediction_test1)
F_measure(data_Test1$TARGET, prediction_test1)